In [7]:
import sys
sys.path.insert(0, '/home/afigueroa')

In [30]:
from transformers import AutoModel, AutoConfig
from tfIntegration.baseModel.models.base import BaseClassificationModel
from tfIntegration.baseModel.training.module import ClassificationModule
import torch
import torch.nn as nn
import pytorch_lightning as pl



╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 from transformers import AutoModel, AutoConfig                                               │
│   2 from tfIntegration.baseModel.models.base import BaseClassificationModel                      │
│ ❱ 3 from tfIntegration.baseModel.training.module import ClassificationModule                     │
│   4 import torch                                                                                 │
│   5 import torch.nn as nn                                                                        │
│   6 import pytorch_lightning as pl                                                               │
│                                                                                                  │
│ /home/afigueroa/tfIntegration/baseModel/training/module.py:2 in <module>                         │
│                                                                                                  │
│    1 from config import CONFIG                                                                   │
│ ❱  2 from processing.datasets import RelationDataset                                             │
│    3                                                                                             │
│    4 import os                                                                                   │
│    5 os.environ["TOKENIZERS_PARALLELISM"] = "false"                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ModuleNotFoundError: No module named 'processing'

In [28]:
from transformers import AutoModel, AutoConfig
from tfIntegration.baseModel.models.base import BaseClassificationModel
from tfIntegration.baseModel.training.module import ClassificationModule
import torch
import torch.nn as nn
import pytorch_lightning as pl

class CombinedModel(nn.Module):
    def __init__(self, base_model_checkpoint, ernie_model_path, dropout=0.05, n_classes=2):
        super(CombinedModel, self).__init__()

        # Load the PyTorch trained base model
        self.base_model = BaseClassificationModel() # Initialize your base model class
        checkpoint = torch.load(base_model_checkpoint) # Load the PyTorch checkpoint
        self.base_model.load_state_dict(checkpoint['state_dict']) # Load the state dict into your base model


        # Load the pre-trained ERNIE model
        ernie_config = AutoConfig.from_pretrained(ernie_model_path)
        self.ernie_model = AutoModel.from_pretrained(ernie_model_path, config=ernie_config)

        # Set the hidden size based on one of the models
        self.hidden_size = ernie_config.hidden_size

        # (combined) model classification head
        self.head = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(self.hidden_size*2, self.hidden_size),
            nn.Tanh(),
            nn.Dropout(dropout),
            nn.Linear(self.hidden_size, n_classes)
        )

    def forward(self, x):
        # Pass input through base model
        base_output = self.base_model(*x)

        # Pass input through ERNIE model
        ernie_output = self.ernie_model(*x)

        # Concatenate the outputs
        combined_output = torch.cat((base_output[0][:, 0, :], ernie_output[0][:, 0, :]), dim=1)

        # Pass through final classification head
        return self.head(combined_output)

In [29]:
base_model_checkpoint = "../baseModel/trained_model/base_model.ckpt"
ernie_model_path = "./models/"
model = CombinedModel(base_model_checkpoint, ernie_model_path)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 base_model_checkpoint = "../baseModel/trained_model/base_model.ckpt"                         │
│   2 ernie_model_path = "./models/"                                                               │
│ ❱ 3 model = CombinedModel(base_model_checkpoint, ernie_model_path)                               │
│   4                                                                                              │
│                                                                                                  │
│ in __init__:14                                                                                   │
│                                                                                                  │
│   11 │   │   # Load the PyTorch trained base model                                               │
│   12 │   │   self.base_model = BaseClassificationModel() # Initialize your base model class      │
│   13 │   │   checkpoint = torch.load(base_model_checkpoint) # Load the PyTorch checkpoint        │
│ ❱ 14 │   │   self.base_model.load_state_dict(checkpoint['state_dict']) # Load the state dict     │
│   15 │   │                                                                                       │
│   16 │   │                                                                                       │
│   17 │   │   # Load the pre-trained ERNIE model                                                  │
│                                                                                                  │
│ /home/afigueroa/tfIntegration/.venv/lib/python3.10/site-packages/torch/nn/modules/module.py:2041 │
│ in load_state_dict                                                                               │
│                                                                                                  │
│   2038 │   │   │   │   │   │   ', '.join('"{}"'.format(k) for k in missing_keys)))               │
│   2039 │   │                                                                                     │
│   2040 │   │   if len(error_msgs) > 0:                                                           │
│ ❱ 2041 │   │   │   raise RuntimeError('Error(s) in loading state_dict for {}:\n\t{}'.format(     │
│   2042 │   │   │   │   │   │   │      self.__class__.__name__, "\n\t".join(error_msgs)))         │
│   2043 │   │   return _IncompatibleKeys(missing_keys, unexpected_keys)                           │
│   2044                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: Error(s) in loading state_dict for BaseClassificationModel:
        Missing key(s) in state_dict: "model.embeddings.position_ids", "model.embeddings.word_embeddings.weight", 
"model.embeddings.position_embeddings.weight", "model.embeddings.token_type_embeddings.weight", 
"model.embeddings.LayerNorm.weight", "model.embeddings.LayerNorm.bias", 
"model.encoder.layer.0.attention.self.query.weight", "model.encoder.layer.0.attention.self.query.bias", 
"model.encoder.layer.0.attention.self.key.weight", "model.encoder.layer.0.attention.self.key.bias", 
"model.encoder.layer.0.attention.self.value.weight", "model.encoder.layer.0.attention.self.value.bias", 
"model.encoder.layer.0.attention.output.dense.weight", "model.encoder.layer.0.attention.output.dense.bias", 
"model.encoder.layer.0.attention.output.LayerNorm.weight", "model.encoder.layer.0.attention.output.LayerNorm.bias",
"model.encoder.layer.0.intermediate.dense.weight", "model.encoder.layer.0.intermediate.dense.bias", 
"model.encoder.layer.0.output.dense.weight", "model.encoder.layer.0.output.